# InstructGoose

> Implementation of Reinforcement Learning from Human Feedback (RLHF) from the InstructGPT paper

Paper: InstructGPT - [Training language models to follow instructions with human feedback](https://arxiv.org/abs/2203.02155)

## Install

```sh
pip install instruct_goose
```

### Resources
I used these resources to implement this

- Copied the `load_yaml` function from https://github.com/Dahoas/reward-modeling
- Learned how to build a dataset to train reward model: https://wandb.ai/carperai/summarize_RLHF/reports/Implementing-RLHF-Learning-to-Summarize-with-trlX--VmlldzozMzAwODM2